<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Restrictions" data-toc-modified-id="Restrictions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Restrictions</a></span><ul class="toc-item"><li><span><a href="#limit" data-toc-modified-id="limit-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>limit</a></span></li><li><span><a href="#distinct" data-toc-modified-id="distinct-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>distinct</a></span><ul class="toc-item"><li><span><a href="#where" data-toc-modified-id="where-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>where</a></span></li></ul></li></ul></li><li><span><a href="#Colonnes" data-toc-modified-id="Colonnes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Colonnes</a></span></li><li><span><a href="#expr" data-toc-modified-id="expr-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>expr</a></span></li><li><span><a href="#Transformations" data-toc-modified-id="Transformations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Transformations</a></span><ul class="toc-item"><li><span><a href="#Projections" data-toc-modified-id="Projections-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Projections</a></span><ul class="toc-item"><li><span><a href="#selectExpr" data-toc-modified-id="selectExpr-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>selectExpr</a></span></li><li><span><a href="#drop" data-toc-modified-id="drop-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>drop</a></span></li><li><span><a href="#orderBy" data-toc-modified-id="orderBy-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>orderBy</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import os
from distutils.sysconfig import get_python_lib,get_python_inc,get_python_version

In [2]:
os.environ['SPARK_HOME']=f'{get_python_lib()}{os.sep}pyspark'
os.environ['PYSPARK_PYTHON']=f'{get_python_lib()}{os.sep}pyspark'.split('Lib')[0]+'python.exe'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [4]:
referentiel = os.path.join(os.getcwd(),'warehouse')
os.makedirs(referentiel, exist_ok=True)

In [5]:
referentiel

'F:\\PythonFormation\\Spark-DataFrames\\warehouse'

In [6]:
spark = SparkSession \
    .builder \
    .appName("PresentationSpark") \
    .config("spark.sql.warehouse.dir", referentiel) \
    .config("spark.executor.cores", 8) \
    .config("spark.executor.memory", '24g') \
    .enableHiveSupport() \
    .getOrCreate()

In [7]:
spark

In [8]:
print(f"spark.executor.cores = {spark.conf.get('spark.executor.cores')}\nspark.executor.memory = {spark.conf.get('spark.executor.memory')}")

spark.executor.cores = 8
spark.executor.memory = 24g


# Restrictions
Une Transformation qui permet de limiter le nombre d’enregistrements du DataFrame résultante est une restriction.

In [9]:
from pyspark.sql.functions import *

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('../donnees/meteo/*/')  #   .cache()


from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('../donnees/postesSynop.csv') #      .cache()

meteoDataFrame.count()

4703265

## limit

In [10]:
meteoDataFrame.selectExpr(
        'numer_sta','t  - 273.15 as temperature'
        ).limit(5).show()

+---------+------------------+
|numer_sta|       temperature|
+---------+------------------+
|     7005|15.400000000000034|
|     7015|15.600000000000023|
|     7020| 16.30000000000001|
|     7027|15.900000000000034|
|     7037|15.600000000000023|
+---------+------------------+



## distinct

In [11]:
meteoDataFrame.select('numer_sta').distinct().count()

62

In [12]:
meteoDataFrame.select('numer_sta').distinct().show(3)

+---------+
|numer_sta|
+---------+
|     7240|
|     7168|
|    61972|
+---------+
only showing top 3 rows



In [13]:
meteoDataFrame.distinct().count()

4528765

### where

In [14]:
villes.where('latitude > 49').show()

+-----+---------------+---------+---------+--------+
|   Id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07005|      ABBEVILLE|   50.136|    1.834|      69|
|07015|  LILLE-LESQUIN|    50.57|   3.0975|      47|
|07020|PTE DE LA HAGUE|49.725166|-1.939833|       6|
|07027| CAEN-CARPIQUET|    49.18|-0.456167|      67|
|07037|     ROUEN-BOOS|   49.383| 1.181667|     151|
|07072|   REIMS-PRUNAY|49.209667| 4.155333|      95|
+-----+---------------+---------+---------+--------+



In [15]:
villes.where('latitude > 49 and altitude > 90').show()

+-----+------------+---------+---------+--------+
|   Id|       ville| latitude|longitude|altitude|
+-----+------------+---------+---------+--------+
|07037|  ROUEN-BOOS|   49.383| 1.181667|     151|
|07072|REIMS-PRUNAY|49.209667| 4.155333|      95|
+-----+------------+---------+---------+--------+



In [16]:
villes.where('latitude > 49 or altitude > 800').show()

+-----+---------------+---------+---------+--------+
|   Id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07005|      ABBEVILLE|   50.136|    1.834|      69|
|07015|  LILLE-LESQUIN|    50.57|   3.0975|      47|
|07020|PTE DE LA HAGUE|49.725166|-1.939833|       6|
|07027| CAEN-CARPIQUET|    49.18|-0.456167|      67|
|07037|     ROUEN-BOOS|   49.383| 1.181667|     151|
|07072|   REIMS-PRUNAY|49.209667| 4.155333|      95|
|07471|  LE PUY-LOUDES|  45.0745|    3.764|     833|
|07591|         EMBRUN|44.565666| 6.502333|     871|
+-----+---------------+---------+---------+--------+



In [17]:
meteoDataFrame.count(), meteoDataFrame.sample(1/100).count(), meteoDataFrame.sample(True,1/100).count()

(4703265, 47102, 46922)

In [18]:
villes.sample(True,2/100,0).show()

+-----+---------------+---------+---------+--------+
|   Id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07168|TROYES-BARBEREY| 48.32467|     4.02|     112|
|07280|  DIJON-LONGVIC|47.267834| 5.088333|     219|
+-----+---------------+---------+---------+--------+



In [19]:
villes.filter("Id == '07168' or Id == '07280'").show()

+-----+---------------+---------+---------+--------+
|   Id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07168|TROYES-BARBEREY| 48.32467|     4.02|     112|
|07280|  DIJON-LONGVIC|47.267834| 5.088333|     219|
+-----+---------------+---------+---------+--------+



In [20]:
villes.filter("Id = '07168' or Id = '07280'").show()

+-----+---------------+---------+---------+--------+
|   Id|          ville| latitude|longitude|altitude|
+-----+---------------+---------+---------+--------+
|07168|TROYES-BARBEREY| 48.32467|     4.02|     112|
|07280|  DIJON-LONGVIC|47.267834| 5.088333|     219|
+-----+---------------+---------+---------+--------+

